In [1]:
import numpy as np
import math
from matplotlib import pyplot as plt
import tensorflow as tf
import random
import pandas as pd
import sys
import gymnasium as gym
%matplotlib qt


In [2]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    


In [3]:
import numpy as np
class rbfbuilder:
    def __init__(self,inpneurons,hiddenneurons,outneurons,minweight=-1,maxweight=1,minc=-1,maxc=1,minvar=0.1,maxvar=1):
        self.inpneurons,self.hiddenneurons,self.outneurons = inpneurons,hiddenneurons,outneurons
        self.weights =np.random.uniform(minweight,maxweight,(hiddenneurons,outneurons))
        self.c = np.random.uniform(minc,maxc,(hiddenneurons,1))
        self.varyans =  np.ones((hiddenneurons,1))
        
    def rbf(self,x, c, sigma):
        inp=-((x - c) / (sigma))**2
        return np.exp(np.clip(inp, -709, 709).astype(float))

    def rbforc(self,x, c, sigma):
        inp=2*((x - c) / (sigma))
        return np.exp(np.clip(inp, -709, 709).astype(float))

    def rbforvar(self,x, c, sigma):
        inp=((x - c)**2 / (sigma)**3)
        return np.exp(np.clip(inp, -709, 709).astype(float))
    
    def train(self,batchinp,batchref,epoch=100,eta=0.1):
        for _ in range(0,epoch):
            self.eta = eta
            
            self.weighteds=[]
            self.hiddenout=[]
            self.errorr=[]
            self.c_hiddenout=[]
            self.var_hiddenout=[]
            self.batchsize = batchinp.shape[0]
            
            
            for u in range(0,self.batchsize):
                hdn=[]
                for a in range(0,self.hiddenneurons):
                    nout=1                    
                    for b in range(0,self.inpneurons):
                        nout*=(self.rbf(batchinp[u,b],self.c[a],self.varyans[a]))
                    hdn.append(nout)
                self.hiddenout.append(hdn)
                
                hdn=[]
                for a in range(0,self.hiddenneurons):
                    nout=1                    
                    for b in range(0,self.inpneurons):
                        nout*=(self.rbforc(batchinp[u,b],self.c[a],self.varyans[a]))
                    hdn.append(nout)
                self.c_hiddenout.append(hdn)
                
                hdn=[]
                for a in range(0,self.hiddenneurons):
                    nout=1                    
                    for b in range(0,self.inpneurons):
                        nout*=(self.rbforvar(batchinp[u,b],self.c[a],self.varyans[a]))
                    hdn.append(nout)
                self.var_hiddenout.append(hdn)
            
            self.weighteds=[np.array(a)*self.weights for a in self.hiddenout]
                    
            self.nnout = [np.sum(a) for a in self.weighteds]
            
            
            for a in range(0,self.batchsize):
                self.errorr.append(batchref[a]-self.nnout[a])

            #update all
            
            
            for u in range(0,self.batchsize):
                self.weights+= self.eta*self.errorr[u]*self.hiddenout[u]
            
            
                
            for u in range(0,self.batchsize):
                self.c+= self.eta*self.errorr[u]*self.weighteds[u] *self.c_hiddenout[u]
                
            for u in range(0,self.batchsize):
                self.varyans+= 2*self.eta*self.errorr[u]*self.weighteds[u] *self.var_hiddenout[u]
            
        return self.nnout



In [4]:

X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])
y = np.array([[0], [1], [1], [0]])
rbff = rbfbuilder(2,20,1)

rbff.train(X,y,epoch=1000,eta=0.01)

[1.821099666149406e-06,
 0.9999983767210147,
 0.9999983767210147,
 8.864118936670329e-07]